In [1]:
# Import Libraries and connect to Twitter API
# Extract I) Big 5 - Openness , SubCategory - Arts

import tweepy
import pandas as pd
import numpy as np


consumer_key = "nZ5pbaBEdbqxRnyAtTwQw5fYz"
consumer_secret = "RiFL5Shi4UMwrZvWQAkUqOJ29WD0WVnVsQIkDWE5El4n918BV0"
access_token = "1499277053031432193-hGvED3N6PyevECoB8xhSo2Rxn7unTB"
access_token_secret = "daL2guXD4xcm3dy1g5rIlaymBdmQ25gyxjEXN1PKuTmNg"


def twitter_setup():
    # Authentication and access using keys:
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)

    # Return API with authentication:
    api = tweepy.API(auth,wait_on_rate_limit=True)
    return api

extractor = twitter_setup()

tweets = tweepy.Cursor(extractor.search_tweets, q="Arts", lang="en",tweet_mode="extended").items(500)

arts_data = pd.DataFrame(data=[[tweet.user.screen_name, tweet.full_text, tweet.user.location] for tweet in tweets],
                    columns=['Name', 'Tweets', 'Location'])

arts_data = arts_data[arts_data['Tweets'].str.contains('Arts')]
dataDF_arts = arts_data
dataDF_arts.to_excel("C:\\Users\\Dell\\Data Mining Project\\openness\\arts\\hashtag_arts_output.xlsx")

In [2]:
# to subset the users in arts category in order to map the same set of users in painting category
# duplicates are removed if there are any

arts_users = dataDF_arts[['Name']]
master_user_DF = arts_users.drop_duplicates(subset=['Name'])
master_user_list = master_user_DF['Name'].tolist()

In [3]:
# Pass the same set of users from master_user_list to extract tweets from paint category
# Remove non-english painting tweets

painting_data_DF = pd.DataFrame()
def get_all_tweets(screen_name):
    
    user_tweet = extractor.user_timeline(screen_name = screen_name, count=200)
    painting_DF = pd.DataFrame(data=[[tweet.user.screen_name,tweet.text,tweet.user.location] for tweet in user_tweet],
                    columns=['Name','Tweets','Location'])
    painting_DF = painting_DF[painting_DF['Tweets'].str.contains('Painting')]
    
    return painting_DF


for user in master_user_list:
    painting_data = get_all_tweets(user)
    painting_data_DF = painting_data_DF.append(painting_data)
    
    
painting_DF_NE = painting_data_DF[painting_data_DF['Tweets'].map(lambda x: x.isascii())]
dataDF_painting = painting_DF_NE
dataDF_painting.to_excel("C:\\Users\\Dell\\Data Mining Project\\openness\\painting\\hashtag_painting_output.xlsx")

In [23]:
# Adding hashtag tweet categories and personalities to the dataframes
# Combine the Arts and Painting dataframe in master_DF

dataDF_arts["Subcategory"] = "Arts"
dataDF_painting["Subcategory"] = "Painting"
master_DF = pd.concat([dataDF_arts, dataDF_painting], ignore_index=False)
# master_DF["Personality"] = "Openness"
master_DF['Tweets'] = master_DF['Tweets'].replace(regex='[^A-Za-z\s]+',value='')
master_DF['Location'] = master_DF['Location'].replace(regex='[^A-Za-z\s]+',value='')
master_DF.to_csv("E:\\Classes Spring 2022\\DM\\final project\\Data\\master_data.csv")
master_DF

<ipython-input-23-59fe87619d24>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataDF_painting["Subcategory"] = "Painting"


,Name,Tweets,Location,Subcategory
5,winterextra,College of Arts and Sciences,Manila,Arts
6,jrhardy2427_jr,RT JadeChung Someone tell my pro wrestling obs...,,Arts
10,UQ_News,Cant stop doomscrolling Look no further \n\nst...,Brisbane,Arts
13,alyna555_,RT CityofSocorro We would like to introduce Ar...,,Arts
15,tedcoeducation,What kind of a person doesnt like pizza \n\nA ...,M Connaught Circus New Delhi,Arts
...,...,...,...,...
489,CcC_8,RT wikivictorian The Fish or The Fish and Flow...,peiron,Arts
496,akira_heichou,RT RennoLeon Devil May Cry Graphic Arts Dant...,Red Grave City,Arts
12,fol_de_rol,ancientsstudies Readers in Paintings httpstcoV...,Olinda Brasil World Travel,Painting
47,fol_de_rol,RT storiadellart Paintings by Henri Martin htt...,Olinda Brasil World Travel,Painting
